# Import Dependencies

In [1]:
# Import our dependencies
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

# Load in CSVs

In [2]:
math_file_path = os.path.join("Student_alcohol_consumption", "student-mat.csv")
math = pd.read_csv(math_file_path)

por_file_path = os.path.join("Student_alcohol_consumption", "student-por.csv")
por = pd.read_csv(por_file_path)

# Preprocessing
###### Very little cleaning needed to be done. Only the grades needed to be bucketed into pass and fail. OneHotEncoder and StandardScaler could handle the rest.

In [3]:
# clean bucket the grades
# 10 and above is a pass
# 9 and below is a fail

def encode_grades(data):
    # bucket the grades into passing(1) and failling(0)
    
    # math
    # failling
    data.loc[(data["G1"] < 10), "G1"] = 0
    data.loc[(data["G2"] < 10), "G2"] = 0
    data.loc[(data["G3"] < 10), "G3"] = 0

    #passing
    data.loc[(data["G1"] >= 10), "G1"] = 1
    data.loc[(data["G2"] >= 10), "G2"] = 1
    data.loc[(data["G3"] >= 10), "G3"] = 1
    
    display(data.head())
    
    return

encode_grades(math)
encode_grades(por)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,0,0,0
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,0,0,0
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,0,0,1
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,1,1,1
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,0,1,1


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,1,1
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,0,1,1
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,1,1,1
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,1,1,1
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,1,1,1


In [4]:
inputs = por.iloc[2]
print(inputs, type(inputs))
test = por.append(inputs)
display(test.tail(1))

school             GP
sex                 F
age                15
address             U
famsize           LE3
Pstatus             T
Medu                1
Fedu                1
Mjob          at_home
Fjob            other
reason          other
guardian       mother
traveltime          1
studytime           2
failures            0
schoolsup         yes
famsup             no
paid               no
activities         no
nursery           yes
higher            yes
internet          yes
romantic           no
famrel              4
freetime            3
goout               2
Dalc                2
Walc                3
health              3
absences            6
G1                  1
G2                  1
G3                  1
Name: 2, dtype: object <class 'pandas.core.series.Series'>


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,1,1,1


# Deep Neural Networks
###### I tried and would of liked to make this look nicer by making a function to handle this so I would not have needed to just copy and paste it 6 times.

In [5]:
def dnn(data, target, drop, file_name):

    # Generate our categorical variable list
    encode_cat = data.dtypes[data.dtypes == "object"].index.tolist()

    # Check the number of unique values in each column
    data[encode_cat].nunique()

    # Create the OneHotEncoder instance
    enc = OneHotEncoder(sparse=False)

    # Fit the encoder and produce encoded DataFrame
    encode_df = pd.DataFrame(enc.fit_transform(data[encode_cat]))

    # Rename encoded columns
    encode_df.columns = enc.get_feature_names(encode_cat)

    # Merge the two DataFrames together and drop the Country column
    data = data.merge(encode_df,left_index=True,right_index=True).drop(encode_cat, 1)
    display(data)

    # Split our preprocessed data into our features and target arrays
    y = data[target].values
    X = data.drop(drop,1).values

    # Split the preprocessed data into a training and testing dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

    # Create a StandardScaler instance
    scaler = StandardScaler()

    # Fit the StandardScaler
    X_scaler = scaler.fit(X_train)

    # Scale the data
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    # Define the model - deep neural net
    number_input_features = len(X_train_scaled[0])
    hidden_nodes_layer1 =  len(X_train_scaled[0]) * 2
    hidden_nodes_layer2 = len(X_train_scaled[0]) * .1

    nn = tf.keras.models.Sequential()

    # First hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

    # Second hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

    # Output layer
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    # Train the model
    fit_model = nn.fit(X_train_scaled,y_train,epochs=50, verbose=0)
    
    # We are going to do a slightly round about method to test our model
    # We are saving and exporting the model then importing it back in
    # This is for two reasons
    # First reason is that we want to save our trained models
    # But we do not need it reimport it to test its accuracy, so why are we doing this?
    # We are testing the imported model because we want to make sure that the model file works
    
    file_path = os.path.join("models", file_name)

    # save model
    nn.save(file_path)
    
    # import model back in
    nn_imported = tf.keras.models.load_model(file_path)

    # Evaluate the model using the test data
    model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test, verbose=0)
    
    return model_accuracy * 100

In [6]:
def target_summary():
    
    G1 = ["G1", "G2", "G3"]
    G2 = ["G2", "G3"]
    G3 = ["G3"]

    summary_table = pd.DataFrame(
        {
            "Math G1": [dnn(math, "G1", G1, "trained_math_G1.h5")],
            "Math G2": [dnn(math, "G2", G2, "trained_math_G2.h5")],
            "Math G3": [dnn(math, "G3", G3, "trained_math_G3.h5")],
            "Portuguese G1": [dnn(por, "G1", G1, "trained_por_G1.h5")],
            "Portuguese G2": [dnn(por, "G2", G2, "trained_por_G2.h5")],
            "Portuguese G3": [dnn(por, "G3", G3, "trained_por_G3.h5")]
        }
    )
    
    # format the accuracy scores to make them easier to read and more descriptive
    summary_table["Math G1"] = summary_table["Math G1"].map("{:.1f}%".format)
    summary_table["Math G2"] = summary_table["Math G2"].map("{:.1f}%".format)
    summary_table["Math G3"] = summary_table["Math G3"].map("{:.1f}%".format)
    summary_table["Portuguese G1"] = summary_table["Portuguese G1"].map("{:.1f}%".format)
    summary_table["Portuguese G2"] = summary_table["Portuguese G2"].map("{:.1f}%".format)
    summary_table["Portuguese G3"] = summary_table["Portuguese G3"].map("{:.1f}%".format)
    
    # change the index name it more clearly state that it is the accuracy scores being displayed
    summary_table = summary_table.rename(index={0: "Accuracy Score"})
    # show us the dataframe
    display(summary_table)
    
    return

In [7]:
target_summary()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
1,17,1,1,1,2,0,5,3,3,1,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,15,1,1,1,2,3,4,3,2,2,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
3,15,4,2,1,3,0,3,2,2,1,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,16,3,3,1,2,0,4,3,2,1,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,20,2,2,1,2,2,5,5,4,4,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
391,17,3,1,2,1,0,2,4,5,3,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
392,21,1,1,1,1,3,5,5,3,3,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
393,18,3,2,3,1,0,4,4,1,3,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
1,17,1,1,1,2,0,5,3,3,1,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,15,1,1,1,2,3,4,3,2,2,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
3,15,4,2,1,3,0,3,2,2,1,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,16,3,3,1,2,0,4,3,2,1,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,20,2,2,1,2,2,5,5,4,4,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
391,17,3,1,2,1,0,2,4,5,3,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
392,21,1,1,1,1,3,5,5,3,3,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
393,18,3,2,3,1,0,4,4,1,3,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
1,17,1,1,1,2,0,5,3,3,1,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,15,1,1,1,2,3,4,3,2,2,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
3,15,4,2,1,3,0,3,2,2,1,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,16,3,3,1,2,0,4,3,2,1,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,20,2,2,1,2,2,5,5,4,4,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
391,17,3,1,2,1,0,2,4,5,3,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
392,21,1,1,1,1,3,5,5,3,3,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
393,18,3,2,3,1,0,4,4,1,3,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
1,17,1,1,1,2,0,5,3,3,1,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,15,1,1,1,2,0,4,3,2,2,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
3,15,4,2,1,3,0,3,2,2,1,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,16,3,3,1,2,0,4,3,2,1,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,19,2,3,1,3,1,5,4,2,1,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
645,18,3,1,1,2,0,4,3,4,1,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
646,18,1,1,2,2,0,1,1,1,1,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
647,17,3,1,2,1,0,2,4,5,3,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
1,17,1,1,1,2,0,5,3,3,1,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,15,1,1,1,2,0,4,3,2,2,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
3,15,4,2,1,3,0,3,2,2,1,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,16,3,3,1,2,0,4,3,2,1,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,19,2,3,1,3,1,5,4,2,1,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
645,18,3,1,1,2,0,4,3,4,1,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
646,18,1,1,2,2,0,1,1,1,1,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
647,17,3,1,2,1,0,2,4,5,3,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
1,17,1,1,1,2,0,5,3,3,1,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,15,1,1,1,2,0,4,3,2,2,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
3,15,4,2,1,3,0,3,2,2,1,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,16,3,3,1,2,0,4,3,2,1,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,19,2,3,1,3,1,5,4,2,1,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
645,18,3,1,1,2,0,4,3,4,1,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
646,18,1,1,2,2,0,1,1,1,1,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
647,17,3,1,2,1,0,2,4,5,3,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


,Math G1,Math G2,Math G3,Portuguese G1,Portuguese G2,Portuguese G3
Accuracy Score,70.7%,82.8%,84.8%,81.0%,84.7%,92.6%


# Conclusion
###### This actually surprised me how high the accuracy is on these models. I thought there would not be nearly enough rows for it to learn from. 
###### On the other hand it is less surprising that the models for the second and third trimesters are so good, because whether or not they passed the previous trimester is a really good predictor. 